# Transformer

Transformer의 핵심 구조인 Multi-head Attention을 구현하는 실습입니다.
1. Multi-head attention 및 self-attention 구현.
2. 각 과정에서 일어나는 연산과 input/output 형태 이해.

### 필요 패키지 import

In [3]:
from torch import nn
from torch.nn import functional as F
from tqdm import tqdm

import torch
import math

## Req. 2-1 Multi-head self-attention 구조 익히기

### **데이터 전처리**
vocab_size 100인 가상의 시퀀스 데이터를 생성합니다. 

각 데이터에 할당된 숫자는 tokenizing과 정수화가 이뤄진 형태입니다.

In [4]:
pad_id = 0
vocab_size = 100

data = [
  [62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75],
  [60, 96, 51, 32, 90],
  [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54],
  [75, 51],
  [66, 88, 98, 47],
  [21, 39, 10, 64, 21],
  [98],
  [77, 65, 51, 77, 19, 15, 35, 19, 23, 97, 50, 46, 53, 42, 45, 91, 66, 3, 43, 10],
  [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34],
  [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43]
]

In [5]:
# 길이 맞춰주기 위해 패딩합니다.
def padding(data):
  max_len = len(max(data, key=len))
  print(f"Maximum sequence length: {max_len}")

  for i, seq in enumerate(tqdm(data)):
    if len(seq) < max_len:
      data[i] = seq + [pad_id] * (max_len - len(seq))

  return data, max_len

In [6]:
data, max_len = padding(data)

Maximum sequence length: 20


100%|██████████| 10/10 [00:00<?, ?it/s]


In [7]:
data

[[62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75, 0, 0, 0, 0],
 [60, 96, 51, 32, 90, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [75, 51, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [66, 88, 98, 47, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [21, 39, 10, 64, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [98, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [77,
  65,
  51,
  77,
  19,
  15,
  35,
  19,
  23,
  97,
  50,
  46,
  53,
  42,
  45,
  91,
  66,
  3,
  43,
  10],
 [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34, 0, 0, 0, 0],
 [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43, 0, 0, 0, 0, 0, 0]]

### Hyperparameter 세팅 및 embedding

In [8]:
d_model = 512  # model의 hidden size
num_heads = 8  # head의 개수

# d_model이 입력을 projection 시킬 임베딩 space의 차원이므로, num_heads로 나누어 떨어져야 한다.

In [9]:
embedding = nn.Embedding(vocab_size, d_model)

# B: batch size, L: maximum sequence length
batch = torch.LongTensor(data)  # (B, L)
batch_emb = embedding(batch)  # (B, L, d_model)

In [10]:
print(batch_emb)
print(batch_emb.shape)

tensor([[[ 2.4039e-02, -2.8913e-01, -8.2269e-01,  ...,  1.3452e-01,
           8.6719e-01, -5.0307e-01],
         [-9.3842e-01,  2.7392e-01, -5.7227e-02,  ..., -8.8614e-01,
           7.2584e-01,  1.4010e-01],
         [-1.2716e+00,  7.6939e-01,  5.6826e-01,  ..., -9.3927e-01,
          -9.6351e-02,  1.0717e+00],
         ...,
         [ 3.2891e-01,  1.3907e+00,  1.2324e-01,  ...,  1.0098e+00,
           8.7211e-01, -5.0238e-01],
         [ 3.2891e-01,  1.3907e+00,  1.2324e-01,  ...,  1.0098e+00,
           8.7211e-01, -5.0238e-01],
         [ 3.2891e-01,  1.3907e+00,  1.2324e-01,  ...,  1.0098e+00,
           8.7211e-01, -5.0238e-01]],

        [[-1.4709e+00,  1.1863e+00,  3.4881e-01,  ..., -5.9714e-02,
           8.5336e-01,  3.3637e-01],
         [ 7.5859e-01,  7.3552e-02,  3.8392e-01,  ..., -9.6302e-02,
          -6.5363e-01,  3.4412e-01],
         [-1.0532e+00, -2.1633e+00, -1.2308e+00,  ...,  1.6122e-01,
          -6.4195e-01,  1.4310e+00],
         ...,
         [ 3.2891e-01,  1

### Linear projection & 여러 head로 나누기

Multi-head attention 내에서 쓰이는 linear projection matrix들을 정의합니다.

In [11]:
w_q = nn.Linear(d_model, d_model)
w_k = nn.Linear(d_model, d_model)
w_v = nn.Linear(d_model, d_model)

In [12]:
w_0 = nn.Linear(d_model, d_model)

In [13]:
q = w_q(batch_emb)  # (B, L, d_model)
k = w_k(batch_emb)  # (B, L, d_model)
v = w_v(batch_emb)  # (B, L, d_model)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 20, 512])
torch.Size([10, 20, 512])
torch.Size([10, 20, 512])


Q, k, v를 `num_head`개의 차원 분할된 여러 vector로 만듭니다.

- 이론적으로는 multi-head attention을 수행하면 input을 각각 다른 head 개수만큼의 Wq, Wk, Wv로 linear transformation 해서 각각 여러번의 attention 수행한 후 concat 한 후 linear transformation 수행해준다
- 구현에서는 Wq, Wk, Wv 한 개씩
- 실제 `attention is all you need` 논문의 구현 예시는 Query vector 한개를 dim으로 쪼개서 진행한다

In [14]:
batch_size = q.shape[0]
d_k = d_model // num_heads

# num_heads * d_k로 쪼갠다
q = q.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
k = k.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
v = v.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 8, 64])


In [15]:
# num_heads를 밖으로 뺌으로써
# 각 head가 (L, d_k) 만큼의 matrix를 가지고 self-attention 수행

q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 8, 20, 64])
torch.Size([10, 8, 20, 64])
torch.Size([10, 8, 20, 64])


### Scaled dot-product self-attention 구현

각 head에서 실행되는 self-attetion 과정입니다.

In [16]:
# shape - (L, L)
# 같은 sequence 내에 서로 다른 token들에게 얼마나 가중치를 두고 attention을 해야하는가
attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # (B, num_heads, L, L)
# softmax - row-wise이기 때문에 dim은 -1
attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

print(attn_dists)
print(attn_dists.shape)

tensor([[[[0.0512, 0.0481, 0.0495,  ..., 0.0583, 0.0583, 0.0583],
          [0.0934, 0.0373, 0.0514,  ..., 0.0275, 0.0275, 0.0275],
          [0.0435, 0.0527, 0.0631,  ..., 0.0376, 0.0376, 0.0376],
          ...,
          [0.0456, 0.0216, 0.0390,  ..., 0.0616, 0.0616, 0.0616],
          [0.0456, 0.0216, 0.0390,  ..., 0.0616, 0.0616, 0.0616],
          [0.0456, 0.0216, 0.0390,  ..., 0.0616, 0.0616, 0.0616]],

         [[0.0558, 0.0386, 0.0357,  ..., 0.0398, 0.0398, 0.0398],
          [0.0366, 0.0220, 0.0631,  ..., 0.0800, 0.0800, 0.0800],
          [0.1042, 0.0313, 0.0192,  ..., 0.0598, 0.0598, 0.0598],
          ...,
          [0.0325, 0.0626, 0.1139,  ..., 0.0492, 0.0492, 0.0492],
          [0.0325, 0.0626, 0.1139,  ..., 0.0492, 0.0492, 0.0492],
          [0.0325, 0.0626, 0.1139,  ..., 0.0492, 0.0492, 0.0492]],

         [[0.0347, 0.0336, 0.0580,  ..., 0.0540, 0.0540, 0.0540],
          [0.0779, 0.0454, 0.0496,  ..., 0.0536, 0.0536, 0.0536],
          [0.0451, 0.0897, 0.1621,  ..., 0

In [17]:
attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

print(attn_values.shape)

torch.Size([10, 8, 20, 64])


### 각 head의 결과물 병합

각 head의 결과물을 concat하고 동일 차원으로 linear projection합니다.

In [18]:
attn_values = attn_values.transpose(1, 2)  # (B, L, num_heads, d_k)
attn_values = attn_values.contiguous().view(batch_size, -1, d_model)  # (B, L, d_model)

print(attn_values.shape)

torch.Size([10, 20, 512])


In [19]:
# w_0 : (d_model, d_model)
# 서로 다른 의미로 foucsing 된 각 head의 self-attention 정보들을 합쳐주는 역할 수행
outputs = w_0(attn_values)

print(outputs)
print(outputs.shape)

tensor([[[ 0.0537,  0.0705,  0.0409,  ..., -0.0230,  0.0024,  0.0823],
         [ 0.0784,  0.0946,  0.0026,  ..., -0.0569,  0.0832,  0.0862],
         [ 0.1182,  0.0166, -0.0293,  ..., -0.0633,  0.0754,  0.0926],
         ...,
         [ 0.0986,  0.0578, -0.0086,  ...,  0.0233,  0.0402,  0.1176],
         [ 0.0986,  0.0578, -0.0086,  ...,  0.0233,  0.0402,  0.1176],
         [ 0.0986,  0.0578, -0.0086,  ...,  0.0233,  0.0402,  0.1176]],

        [[ 0.1813,  0.0323, -0.1066,  ..., -0.0165, -0.2181,  0.2451],
         [ 0.1427,  0.0957, -0.1201,  ..., -0.0217, -0.1856,  0.2232],
         [ 0.1673,  0.0554, -0.0820,  ...,  0.0168, -0.1691,  0.1937],
         ...,
         [ 0.1713,  0.0637, -0.1153,  ...,  0.0086, -0.1351,  0.1935],
         [ 0.1713,  0.0637, -0.1153,  ...,  0.0086, -0.1351,  0.1935],
         [ 0.1713,  0.0637, -0.1153,  ...,  0.0086, -0.1351,  0.1935]],

        [[ 0.0257, -0.0709, -0.0801,  ...,  0.0526, -0.0953,  0.1171],
         [ 0.0005, -0.0569, -0.0840,  ..., -0

## Req. 2-2 Multi-head self-attention 모듈 클래스 구현

위의 과정을 모두 합쳐 하나의 Multi-head attention 모듈 class를 구현하겠습니다.

아래 코드의 TODO 부분을 채워주세요.

In [20]:
class MultiheadAttention(nn.Module):
  def __init__(self):
    super(MultiheadAttention, self).__init__()

    # Q, K, V learnable matrices
    self.w_q = nn.Linear(d_model, d_model)
    self.w_k = nn.Linear(d_model, d_model)
    self.w_v = nn.Linear(d_model, d_model)

    # Linear projection for concatenated outputs
    self.w_0 = nn.Linear(d_model, d_model)

  # scaled-dot product attention
  def self_attention(self, q, k, v):
    attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # (B, num_heads, L, L)
    attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

    attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

    return attn_values

  def forward(self, q, k, v):
    batch_size = q.shape[0]

    # linear projection
    ################################################################################
    # TODO 1: Implement the forward pass for linear projection.                #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    
    q=self.w_q(q)
    k=self.w_k(k)
    v=self.w_v(v)
    
    # head만큼 쪼개준다
    ################################################################################
    # TODO 2: Implement the forward pass for split head.                #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    q=q.view(batch_size,-1,num_heads,d_k)
    k=k.view(batch_size,-1,num_heads,d_k)
    v=v.view(batch_size,-1,num_heads,d_k)
    
    # 각 head가 (L, d_k)의 matrix를 담당하도록 만든다
    q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
    k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
    v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

    attn_values = self.self_attention(q, k, v)  # (B, num_heads, L, d_k)
    attn_values = attn_values.transpose(1, 2).contiguous().view(batch_size, -1, d_model)  # (B, L, num_heads, d_k) => (B, L, d_model)

    return self.w_0(attn_values)

In [21]:
multihead_attn = MultiheadAttention()

outputs = multihead_attn(batch_emb, batch_emb, batch_emb)  # (B, L, d_model)

In [22]:
print(outputs)
print(outputs.shape)  # (batch_size, length, d_model)

tensor([[[ 1.6090e-01, -1.6496e-03,  9.4370e-02,  ..., -6.2381e-02,
           1.1788e-01, -1.7284e-01],
         [-1.4018e-01, -6.1340e-02,  1.6322e-01,  ..., -2.3620e-01,
           1.3613e-02,  5.0560e-02],
         [-1.9657e-01,  2.6002e-02,  7.0686e-02,  ..., -3.6916e-02,
           5.6904e-02, -1.0916e-01],
         ...,
         [-1.7990e-01,  2.4116e-01,  2.4578e-01,  ..., -6.7102e-02,
           1.1746e-01, -3.7070e-02],
         [-1.7990e-01,  2.4116e-01,  2.4578e-01,  ..., -6.7102e-02,
           1.1746e-01, -3.7070e-02],
         [-1.7990e-01,  2.4116e-01,  2.4578e-01,  ..., -6.7102e-02,
           1.1746e-01, -3.7070e-02]],

        [[-1.5282e-01,  4.1749e-01,  1.2376e-01,  ..., -1.4899e-01,
          -1.0264e-01,  6.8223e-02],
         [-7.5518e-02,  1.4284e-01,  5.3329e-02,  ..., -2.3309e-02,
          -5.6342e-02,  9.0262e-02],
         [ 8.7592e-03,  2.9211e-02,  2.8965e-02,  ...,  4.5807e-03,
           9.0758e-02,  1.0392e-02],
         ...,
         [-6.5287e-01,  7